In [4]:
import typing as tp
import httpx
import json
import re


In [5]:
DUCK_DUCK_GO_URL = "https://duckduckgo.com/duckchat/v1/chat"

In [25]:
class LLM:
    def __init__(
        self, 
        llm_chat_url: str, 
        model: tp.Literal[
            "claude-3-haiku-20240307",
            "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            "mistralai/Mixtral-8x7B-Instruct-v0.1",
            "gpt-4o-mini"
        ] = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
        system_prompt: str = ""
    ):
        self.url = llm_chat_url
        self.model = model
        self.system_prompt = system_prompt
        
        self.client = httpx.Client(headers={
            "accept": "text/event-stream",
            "accept-language": "en-US,en;q=0.9,ru-RU;q=0.8,ru;q=0.7",
            "content-type": "application/json",
            "cookie": "dcm=5",
            "dnt": "1",
            "origin": "https://duckduckgo.com",
            "priority": "u=1, i",
            "referer": "https://duckduckgo.com/",
            "sec-ch-ua": '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"macOS"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-origin",
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36",
            "x-vqd-4": "4-248234416339843192477661751153195587515",
        })

    def _assemble_body(self, prompt: str):
        return {
            "model": self.model,
            "messages": [
                {
                    "role": "system",
                    "content": self.system_prompt
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
        }

    def __parse_chunk(self, chunk: str) -> str :
        if len(chunk) == 0:
            return ""
        payload = chunk[5:]
        try:
            return json.loads(payload)["message"]
        except Exception as e:
            print(e, chunk)
            return ""
        
    def prompt(self, prompt) -> str:
        response = ""
        
        with self.client.stream("POST", self.url,  json=self._assemble_body(prompt), timeout=120) as r:
            for chunk in r.iter_lines():
                new_data = self.__parse_chunk(chunk)
                response += new_data

        return response

In [26]:
llama70b = LLM(
    llm_chat_url=DUCK_DUCK_GO_URL,
    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
    system_prompt=""
)

In [27]:
response = llama70b.prompt(
    prompt="Привет, как дела!?"
)

print(response)

Expecting value: line 1 column 3 (char 2) data: [DONE]
Привет! Хорошо, спасибо. Как у вас дела?
